In [1]:
%matplotlib widget

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from ProcessResults import *
from DatasetTester import *
from GetLandmarks_og import GetLandmarks 
#from LandmarkTester_alt import LandmarkTester

from landmark_utils import vis_landmarks, compare_landmarks
from vis_utils import *


import cv2
import numpy as np
import time
import timeit
from pathlib import Path
import csv
import pickle

from autosklearn.experimental.askl2 import AutoSklearn2Classifier

from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy import linalg
import seaborn as sns

ModuleNotFoundError: No module named 'dlib'

In [ ]:
MEEI_path = Path('./MEEI_Standard_Set')
pat_info = pd.read_excel(MEEI_path/"pat_info.xlsx", sheet_name='Sheet1')

: 

In [ ]:
pat_info = pd.read_pickle('MEEI_pat_info.pkl')

with open('all_pats_processed_data.pickle','rb') as f:
    loaded_pats = pickle.load(f)

: 

In [ ]:
pat_info

: 

In [ ]:
# x - choose one
all_ids = loaded_pats[0]
all_imgs = loaded_pats[1]
all_lms = loaded_pats[2]
all_features = loaded_pats[3]

# y
all_hb = loaded_pats[4]

: 

In [ ]:
# Find NaN and Inf values
bad_indices = np.where(np.isnan(all_features).sum(axis=1) > 0)[0]
bad_indices = np.append(bad_indices, np.where(np.isinf(all_features).sum(axis=1) > 0)[0])
bad_indices.sort()
bad_indices

: 

In [ ]:
# drop bad indices 
# x
all_ids = np.delete(all_ids, bad_indices, axis=0)
all_imgs = np.delete(all_imgs, bad_indices, axis=0)
all_lms = np.delete(all_lms, bad_indices, axis=0)
all_features = np.delete(all_features, bad_indices, axis=0)

# y
all_hb = np.delete(all_hb, bad_indices, axis=0)

all_features.shape, all_hb.shape

: 

In [ ]:
plot_df = pd.DataFrame(all_features, columns = ['f'+str(i) for i in range(0, 29)])

: 

In [ ]:
plot_df['hb'] = all_hb

: 

In [ ]:
def feature_plot(df, which_feature, group_by):
    sns.displot(df, x=which_feature, hue=group_by, element="step")

#for i in range(0, 29):
#    feature_plot(plot_df, 'f'+str(i), 'hb')

: 

In [ ]:
# Split IDs into train and test
random.seed(85)
subject_ids = [i for i in range(51)]
train_ids = np.sort(np.array(random.sample(subject_ids, 44)))
test_ids = np.sort(np.array([i for i in subject_ids if i not in train_ids]))

print('Training IDs:',train_ids)
print('Testing IDs:',test_ids)

: 

In [ ]:
# Next, match the IDs to the indices in the data
train_idx = np.sort(np.where(np.isin(all_ids, train_ids))[0])
test_idx = np.sort(np.where(np.isin(all_ids, test_ids))[0])

x_train, x_test = all_features[train_idx], all_features[test_idx]
y_train, y_test = all_hb[train_idx], all_hb[test_idx]

print('Training x shape:', x_train.shape)
print('Training y shape:', y_train.shape)
print('Testing x shape:', x_test.shape)
print('Testing y shape:', y_test.shape)

: 

In [ ]:
x_train[0], y_train[0]

: 

In [ ]:
x_test[0], y_test[0]

: 

In [ ]:
y_train[0:10]

: 

In [ ]:
# Label encoding changes 1-6 to 0-5
le = LabelEncoder()
y_train = le.fit_transform(y_train)

: 

In [ ]:
y_train[0:10]

: 

In [ ]:
cls = AutoSklearn2Classifier()
cls.fit(x_train,y_train)
y_pred = cls.predict(x_test)

: 

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %2f" % (rmse))
print('Accuracy:', round(accuracy_score(y_test, y_pred), 3))

: 

In [ ]:
y_pred[20:30], y_test[20:30]

: 

: 

### Now try data standardaization / normalization

: 

: 

: 